# ECOSTRESS Evapotranspiration Data Visualization

This notebook explores and visualizes ECOSTRESS (ECOsystem Spaceborne Thermal Radiometer Experiment on Space Station) evapotranspiration data downloaded from AppEEARS.

**Dataset**: ECOSTRESS L3T JET (Jet Propulsion Laboratory Evapotranspiration) - Daily ET estimates

**Data Source**: NASA AppEEARS (Application for Extracting and Exploring Analysis Ready Samples)

**Goals**:
- Load and inspect the ECOSTRESS GeoTIFF data structure
- Visualize daily evapotranspiration patterns
- Understand the spatial extent and resolution of the data
- Create publication-ready maps of ET estimates

---

## 1. Environment Setup

**Purpose:** Import required libraries and configure the analysis environment.

**Libraries used:**
- `rasterio`: Reading and writing geospatial raster data (GeoTIFF files)
- `xarray` + `rioxarray`: N-dimensional array handling with geospatial extensions
- `matplotlib`: Visualization and plotting
- `numpy`: Numerical operations
- `pathlib`: Cross-platform file path handling

In [ ]:
# Core libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from pathlib import Path
import warnings

# Geospatial libraries
import rasterio
from rasterio.plot import show
import xarray as xr
import rioxarray

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

print("Libraries loaded successfully!")

---

## 2. Data Path Configuration

**Purpose:** Define paths to the ECOSTRESS data and verify file accessibility.

**File naming convention:** `ECOv002_L3T_JET_{orbit}_{scene}_{tile}_{datetime}_{build}_{version}_ETdaily.tif`
- `ECOv002`: ECOSTRESS Version 2 product
- `L3T`: Level 3 Tiled product
- `JET`: JPL Evapotranspiration algorithm
- `ETdaily`: Daily evapotranspiration estimate

In [ ]:
# Define data directory path
data_dir = Path("../../data/raw/ECOSTRESS")

print(f"Looking for ECOSTRESS data in: {data_dir.absolute()}")
print(f"Directory exists: {data_dir.exists()}")
print()

# List available ECOSTRESS files
if data_dir.exists():
    tif_files = list(data_dir.glob("*.tif"))
    print(f"Found {len(tif_files)} GeoTIFF file(s):")
    for f in tif_files:
        print(f"  - {f.name}")
else:
    print("Data directory not found. Check the path.")

In [ ]:
# Select the ECOSTRESS ET file
ecostress_file = data_dir / "ECOv002_L3T_JET_28282_006_15TVG_20230702T130422_0712_01_ETdaily.tif"

print(f"Selected file: {ecostress_file.name}")
print(f"File exists: {ecostress_file.exists()}")

# Parse metadata from filename
filename_parts = ecostress_file.stem.split('_')
print("\nParsed filename metadata:")
print(f"  Product version: {filename_parts[0]}")
print(f"  Processing level: {filename_parts[1]}")
print(f"  Algorithm: {filename_parts[2]} (JPL Evapotranspiration)")
print(f"  Orbit number: {filename_parts[3]}")
print(f"  Scene: {filename_parts[4]}")
print(f"  Tile ID: {filename_parts[5]}")
print(f"  Acquisition datetime: {filename_parts[6]}")
print(f"  Variable: {filename_parts[9]} (Daily Evapotranspiration)")

---

## 3. Load and Inspect Data Structure

**Purpose:** Open the GeoTIFF file and examine its structure, dimensions, and metadata.

**What we'll learn:**
- Spatial dimensions (rows x columns)
- Coordinate Reference System (CRS)
- Geospatial bounds (extent)
- Pixel resolution
- Data type and NoData values

In [ ]:
# Open with rasterio to inspect metadata
with rasterio.open(ecostress_file) as src:
    print("=" * 60)
    print("ECOSTRESS GeoTIFF Metadata")
    print("=" * 60)
    print(f"\nDimensions:")
    print(f"  Width: {src.width} pixels")
    print(f"  Height: {src.height} pixels")
    print(f"  Bands: {src.count}")
    
    print(f"\nCoordinate Reference System:")
    print(f"  CRS: {src.crs}")
    print(f"  CRS WKT: {src.crs.wkt[:100]}..." if src.crs else "  CRS: Not defined")
    
    print(f"\nSpatial Bounds:")
    print(f"  Left (West): {src.bounds.left:.4f}")
    print(f"  Right (East): {src.bounds.right:.4f}")
    print(f"  Bottom (South): {src.bounds.bottom:.4f}")
    print(f"  Top (North): {src.bounds.top:.4f}")
    
    print(f"\nPixel Resolution:")
    print(f"  X resolution: {src.res[0]:.6f} units")
    print(f"  Y resolution: {src.res[1]:.6f} units")
    
    print(f"\nData Type: {src.dtypes[0]}")
    print(f"NoData Value: {src.nodata}")
    
    # Store transform for later use
    transform = src.transform
    crs = src.crs

In [ ]:
# Load data using rioxarray for xarray integration
da = rioxarray.open_rasterio(ecostress_file)

print("xarray DataArray structure:")
print(da)
print("\nDimensions:", da.dims)
print("Sizes:", dict(da.sizes))
print("Coordinates:", list(da.coords))

---

## 4. Data Quality Assessment

**Purpose:** Examine data statistics and identify valid/invalid pixels.

**Key checks:**
- Distribution of ET values
- Percentage of valid (non-NoData) pixels
- Value ranges and outliers
- Data quality flags (if present)

In [ ]:
# Extract the data array (squeeze to remove band dimension if single band)
et_data = da.squeeze()

# Get NoData value
nodata_val = da.rio.nodata
print(f"NoData value: {nodata_val}")

# Mask NoData values
et_masked = et_data.where(et_data != nodata_val)

# Calculate statistics
print("\n" + "=" * 60)
print("Evapotranspiration Data Statistics")
print("=" * 60)

total_pixels = et_data.size
valid_pixels = et_masked.count().values
invalid_pixels = total_pixels - valid_pixels

print(f"\nPixel Coverage:")
print(f"  Total pixels: {total_pixels:,}")
print(f"  Valid pixels: {valid_pixels:,} ({100*valid_pixels/total_pixels:.1f}%)")
print(f"  NoData pixels: {invalid_pixels:,} ({100*invalid_pixels/total_pixels:.1f}%)")

print(f"\nET Value Statistics (mm/day):")
print(f"  Minimum: {float(et_masked.min()):.4f}")
print(f"  Maximum: {float(et_masked.max()):.4f}")
print(f"  Mean: {float(et_masked.mean()):.4f}")
print(f"  Std Dev: {float(et_masked.std()):.4f}")
print(f"  Median: {float(et_masked.median()):.4f}")

In [ ]:
# Create histogram of ET values
fig, ax = plt.subplots(figsize=(10, 6))

# Flatten and remove NaN values for histogram
valid_values = et_masked.values.flatten()
valid_values = valid_values[~np.isnan(valid_values)]

ax.hist(valid_values, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
ax.axvline(np.mean(valid_values), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(valid_values):.2f} mm/day')
ax.axvline(np.median(valid_values), color='orange', linestyle='--', linewidth=2, label=f'Median: {np.median(valid_values):.2f} mm/day')

ax.set_xlabel('Evapotranspiration (mm/day)')
ax.set_ylabel('Frequency (pixels)')
ax.set_title('Distribution of Daily Evapotranspiration Values\nECOSTRESS L3T JET - July 2, 2023')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 5. Basic Visualization

**Purpose:** Create initial visualizations of the ECOSTRESS evapotranspiration data.

**What we'll create:**
- Simple raster plot using rasterio
- Customized map with appropriate colormap for ET data
- Comparison of different visualization approaches

In [ ]:
# Quick visualization using rasterio's show function
fig, ax = plt.subplots(figsize=(12, 10))

with rasterio.open(ecostress_file) as src:
    show(src, ax=ax, title='ECOSTRESS Daily ET (Quick View)')

plt.tight_layout()
plt.show()

In [ ]:
# Create a more customized visualization
fig, ax = plt.subplots(figsize=(14, 12))

# Plot the masked data with a suitable colormap for ET
# YlGnBu is good for water-related variables (yellow=low, blue=high)
im = et_masked.plot(
    ax=ax,
    cmap='YlGnBu',
    robust=True,  # Use 2nd and 98th percentiles for color scaling
    cbar_kwargs={
        'label': 'Evapotranspiration (mm/day)',
        'shrink': 0.8,
        'aspect': 30
    }
)

ax.set_title('ECOSTRESS Daily Evapotranspiration\nJuly 2, 2023 | Tile 15TVG', fontsize=16)
ax.set_xlabel('Easting (m)')
ax.set_ylabel('Northing (m)')

# Add gridlines
ax.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

---

## 6. Enhanced Visualization with Custom Colormap

**Purpose:** Create publication-quality maps with appropriate styling for evapotranspiration data.

**Visualization choices:**
- Custom colormap optimized for ET values (brown=dry/low ET, green/blue=high ET)
- Proper handling of NoData regions
- Informative title and labels
- Scale bar and north arrow (if coordinates allow)

In [ ]:
# Create a custom ET colormap (brown -> yellow -> green -> blue)
colors_et = ['#8B4513', '#D2691E', '#F4A460', '#FFFF00', '#ADFF2F', '#32CD32', '#228B22', '#006400', '#00CED1', '#1E90FF']
et_cmap = LinearSegmentedColormap.from_list('et_custom', colors_et, N=256)

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Left plot: YlGnBu colormap
et_masked.plot(
    ax=axes[0],
    cmap='YlGnBu',
    robust=True,
    cbar_kwargs={'label': 'ET (mm/day)', 'shrink': 0.7}
)
axes[0].set_title('YlGnBu Colormap\n(Standard Water Variable)', fontsize=12)
axes[0].set_xlabel('Easting (m)')
axes[0].set_ylabel('Northing (m)')

# Right plot: Custom ET colormap
et_masked.plot(
    ax=axes[1],
    cmap=et_cmap,
    robust=True,
    cbar_kwargs={'label': 'ET (mm/day)', 'shrink': 0.7}
)
axes[1].set_title('Custom ET Colormap\n(Brown=Low, Blue=High)', fontsize=12)
axes[1].set_xlabel('Easting (m)')
axes[1].set_ylabel('Northing (m)')

plt.suptitle('ECOSTRESS Daily Evapotranspiration - July 2, 2023\nColormap Comparison', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Publication-quality figure
fig, ax = plt.subplots(figsize=(12, 12))

# Calculate percentiles for better visualization
vmin = float(et_masked.quantile(0.02))
vmax = float(et_masked.quantile(0.98))

# Plot with fixed color limits
im = ax.imshow(
    et_masked.values,
    cmap='YlGnBu',
    vmin=vmin,
    vmax=vmax,
    extent=[
        float(et_masked.x.min()),
        float(et_masked.x.max()),
        float(et_masked.y.min()),
        float(et_masked.y.max())
    ],
    origin='upper'
)

# Add colorbar
cbar = plt.colorbar(im, ax=ax, shrink=0.8, aspect=30, pad=0.02)
cbar.set_label('Evapotranspiration (mm/day)', fontsize=12)

# Labels and title
ax.set_xlabel('Easting (m)', fontsize=12)
ax.set_ylabel('Northing (m)', fontsize=12)
ax.set_title('ECOSTRESS L3T JET Daily Evapotranspiration\nAcquisition: July 2, 2023 13:04:22 UTC | Tile: 15TVG', 
             fontsize=14, fontweight='bold')

# Add text box with statistics
stats_text = f'Statistics:\nMean: {float(et_masked.mean()):.2f} mm/day\nMax: {float(et_masked.max()):.2f} mm/day\nValid pixels: {valid_pixels:,}'
props = dict(boxstyle='round', facecolor='white', alpha=0.8)
ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=props)

plt.tight_layout()
plt.show()

---

## 7. Spatial Context and Coordinate Information

**Purpose:** Understand the geographic context of the data including the tile location and coordinate system.

**ECOSTRESS Tiling System:**
- Uses the Military Grid Reference System (MGRS) / Sentinel-2 tiling scheme
- Tile 15TVG corresponds to a specific 110km x 110km area
- Typical resolution is 70m for ECOSTRESS products

In [ ]:
print("=" * 60)
print("Coordinate Reference System Details")
print("=" * 60)

print(f"\nCRS: {da.rio.crs}")
print(f"\nSpatial Extent:")
print(f"  X (Easting): {float(da.x.min()):.2f} to {float(da.x.max()):.2f}")
print(f"  Y (Northing): {float(da.y.min()):.2f} to {float(da.y.max()):.2f}")

# Calculate approximate center
center_x = (float(da.x.min()) + float(da.x.max())) / 2
center_y = (float(da.y.min()) + float(da.y.max())) / 2
print(f"\nApproximate Center (projected):")
print(f"  X: {center_x:.2f}")
print(f"  Y: {center_y:.2f}")

# Pixel size
pixel_size_x = abs(float(da.x[1] - da.x[0]))
pixel_size_y = abs(float(da.y[1] - da.y[0]))
print(f"\nPixel Size:")
print(f"  X resolution: {pixel_size_x:.2f} m")
print(f"  Y resolution: {pixel_size_y:.2f} m")

# Coverage area
coverage_x = (float(da.x.max()) - float(da.x.min())) / 1000  # km
coverage_y = (float(da.y.max()) - float(da.y.min())) / 1000  # km
print(f"\nSpatial Coverage:")
print(f"  X extent: {coverage_x:.2f} km")
print(f"  Y extent: {coverage_y:.2f} km")
print(f"  Total area: {coverage_x * coverage_y:.2f} km²")

---

## 8. Export Visualization

**Purpose:** Save the visualization to a file for use in presentations or publications.

**Output formats:**
- PNG: High-resolution raster image
- PDF: Vector format for publications (optional)

In [ ]:
# Create output directory for figures
figures_dir = Path("../../figures/ecostress")
figures_dir.mkdir(parents=True, exist_ok=True)

# Create and save publication figure
fig, ax = plt.subplots(figsize=(12, 12), dpi=150)

im = ax.imshow(
    et_masked.values,
    cmap='YlGnBu',
    vmin=vmin,
    vmax=vmax,
    extent=[
        float(et_masked.x.min()),
        float(et_masked.x.max()),
        float(et_masked.y.min()),
        float(et_masked.y.max())
    ],
    origin='upper'
)

cbar = plt.colorbar(im, ax=ax, shrink=0.8, aspect=30, pad=0.02)
cbar.set_label('Evapotranspiration (mm/day)', fontsize=12)

ax.set_xlabel('Easting (m)', fontsize=12)
ax.set_ylabel('Northing (m)', fontsize=12)
ax.set_title('ECOSTRESS L3T JET Daily Evapotranspiration\nAcquisition: July 2, 2023 13:04:22 UTC | Tile: 15TVG', 
             fontsize=14, fontweight='bold')

stats_text = f'Statistics:\nMean: {float(et_masked.mean()):.2f} mm/day\nMax: {float(et_masked.max()):.2f} mm/day\nValid pixels: {valid_pixels:,}'
props = dict(boxstyle='round', facecolor='white', alpha=0.8)
ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=props)

plt.tight_layout()

# Save figure
output_path = figures_dir / "ecostress_et_20230702.png"
plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
print(f"Figure saved to: {output_path}")

plt.show()

---

## 9. Summary and Next Steps

**What we accomplished:**
1. Successfully loaded ECOSTRESS L3T JET evapotranspiration data from AppEEARS
2. Examined the data structure, dimensions, and coordinate system
3. Assessed data quality including valid pixel coverage and value distributions
4. Created multiple visualizations with different colormaps
5. Exported a publication-quality figure

**Key findings:**
- The data uses UTM projection with 70m resolution
- Daily ET values typically range from 0 to several mm/day
- Spatial patterns reflect vegetation and land cover differences

**Potential next steps:**
- Reproject to lat/lon (WGS84) for comparison with other datasets
- Overlay with land cover data to analyze ET by vegetation type
- Compare with SIF data to understand photosynthesis-ET relationships
- Time series analysis with multiple ECOSTRESS acquisitions
- Validate against ground-based flux tower measurements

In [ ]:
# Final summary
print("=" * 60)
print("ECOSTRESS Analysis Summary")
print("=" * 60)
print(f"\nFile: {ecostress_file.name}")
print(f"Date: July 2, 2023")
print(f"Product: L3T JET (Daily Evapotranspiration)")
print(f"Tile: 15TVG")
print(f"\nData dimensions: {da.shape}")
print(f"Valid pixels: {valid_pixels:,} ({100*valid_pixels/total_pixels:.1f}%)")
print(f"Mean ET: {float(et_masked.mean()):.3f} mm/day")
print(f"\nAnalysis complete!")